In [20]:
import socket
import yaml
import argparse
from protobuf_to_dict import protobuf_to_dict, dict_to_protobuf

from ecat_repl import ZmsgIO
from ecat_repl import CtrlCmd
from ecat_repl import SdoCmd
from ecat_repl import (
    ctrl_cmd_start,
    ctrl_cmd_stop,
    ctrl_cmd_fan,
    ctrl_cmd_led,
    ctrl_cmd_set_home,
    ctrl_cmd_set_zero,
    ctrl_cmd_set_min_pos,
    ctrl_cmd_set_max_pos,
)

#uri = socket.gethostname()+".local:5555"
uri = "com-exp.wrt:5555"
#uri = "localhost:5555"
io = ZmsgIO(uri)

bId = 1
ids = [bId]

homing = {"type": "TRJ_CMD",
          "trajectory_cmd": {"type": "HOMING", "name": "homing", "board_id": bId,
                             "homing_par": {"x": [0,3]}}}

xs = [0,1,2,3,4]
xs = [x*2 for x in xs] 
smooth = {"type": "TRJ_CMD",
             "trajectory_cmd": {"type": "SMOOTHER", "name": "smooth", "board_id": bId,
                                "smooth_par": {"x": xs, "y": [0, 0.5, 0, -0.5, 0]}}} 
sine = {"type": "TRJ_CMD",
        "trajectory_cmd": {"type": "SINE", "name": "sine", "board_id": bId,
                           "period_par": {"freq": 0.2, "ampl": 2.79, "teta": 0, "secs": 500}}}

add_homing = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing"]}}
add_smooth = {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["smooth"]}}
add_sine =   {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["sine"]}}
add_all =   {"type": "TRJ_QUEUE_CMD",
              "trj_queue_cmd": {"type": "PUSH_QUE", "trj_names": ["homing","smooth","sine"]}}

io.debug=0

In [21]:
reply = io.doit(ctrl_cmd_set_home.set_bid(bId))

In [22]:
reply = io.doit(ctrl_cmd_start.set_bid(bId).set_value(0x3B).set_gains(u'POSITION',200,0,15,0,0,0))

In [23]:
# define trajectories
reply = io.doit(homing)
reply = io.doit(smooth)
reply = io.doit(sine)

In [24]:
#reply = io.doit(add_homing)
#reply = io.doit(add_smooth)
#reply = io.doit(add_sine)
reply = io.doit(add_all)

In [25]:
reply = io.doit(ctrl_cmd_stop.set_bid(bId))

In [2]:
reply = io.doit(ctrl_cmd_start.set_bid(bId).set_value(0xD4))

In [6]:
io.debug=1
gains = {}
gains['type'] = "IMPEDANCE"
gains['kp'] = 1
gains['kd'] = 0
gains['ki'] = 0
gains['kkp'] = 5
gains['kkd'] = 0.01
gains['kki'] = 0.8
reply = io.doit(ctrl_cmd_start.set_bid(bId).set_value(0xD4).set_gains(*gains.values()))

{'ctrl_cmd': {'type': 'CTRL_CMD_START', 'board_id': 1, 'value': 212, 'gains': {'type': 'IMPEDANCE', 'kp': 1, 'ki': 0, 'kd': 0, 'kkp': 5, 'kki': 0.01, 'kkd': 0.8}}, 'type': 'CTRL_CMD'}
type: CTRL_CMD
ctrl_cmd {
  type: CTRL_CMD_START
  board_id: 1
  value: 212.0
  gains {
    type: IMPEDANCE
    kp: 1.0
    ki: 0.0
    kd: 0.0
    kkp: 5.0
    kki: 0.01
    kkd: 0.8
  }
}



In [17]:
cmd

NameError: name 'cmd' is not defined

In [14]:
xs

[0, 2, 4, 6, 8]

In [5]:
gains.values()

dict_values(['IMPEDANCE', 1, 0, 0, 5, 0.01, 0.8])